In [1]:
import os
import pandas as pd
from datetime import datetime as dt 
from datetime import date
import string
import math as mt
import numpy as np
from pandasql import sqldf

In [2]:
os.chdir('C:\\Users\\prajwal.d1\\Desktop\\Pendency')
RTO_RVP=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\totalShipmentsPendingInTheHub_2025-Apr-19_09.csv")
UT=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\untraceableShipments_2025-Apr-19_05.csv")

C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_3204\4252779536.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  RTO_RVP=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\totalShipmentsPendingInTheHub_2025-Apr-19_09.csv")


In [4]:
Map = pd.read_csv(r"C:\Users\prajwal.d1\Desktop\Pendency\ODH Model pincode Tracker.csv")
New1 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\FMA'25 FA Accounting Data 18.04.25.xlsb",engine="pyxlsb", sheet_name="Raw Data")  # FA Accounting Data
New2 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\NFA-18.04.25.xlsb", engine="pyxlsb", sheet_name="RAW")  # NFA Data
New3 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\Gunti_Raw2025-04-19.xlsx")  # Overall >3days

In [5]:
Map=Map[['ODH/MDH Name','Vendor']]
Map.columns=['HubName','Vendor Name']
Map['HubName']=Map['HubName'].apply(lambda x:x.upper())
Map['Vendor Name']=Map['Vendor Name'].apply(lambda x:x.upper())
Map=Map.drop_duplicates()
RTO_RVP['HubName']=RTO_RVP['HubName'].apply(lambda x:x.upper())
RTO_RVP=RTO_RVP.merge(Map,how='left',on=['HubName'])
RTO_RVP=RTO_RVP.dropna(subset=['Vendor Name'])

In [6]:
RTO_RVP['RefTime']=np.where(RTO_RVP['TypeOfShipment']=='RTO',RTO_RVP['RTOCreatedDateTime'],RTO_RVP['LastUpdatDateTime'])
RTO_RVP=RTO_RVP[(RTO_RVP['RefTime']!='---')]
RTO_RVP['TimeDelta']=dt.now()-pd.to_datetime(RTO_RVP['RefTime'])
RTO_RVP['Ageing']=RTO_RVP['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))

In [7]:
RTO_RVP['Month']=pd.DatetimeIndex(RTO_RVP['RefTime']).month
RTO_RVP['Year']=pd.DatetimeIndex(RTO_RVP['RefTime']).year
RTO_RVP=RTO_RVP.drop(['RefTime','TimeDelta'],axis=1)
RTO_RVP['Month']=RTO_RVP['Month'].astype(str)
RTO_RVP['Length']=RTO_RVP['Month'].apply(lambda x:len(x))
RTO_RVP['Month']=np.where(RTO_RVP['Length']==2,RTO_RVP['Year'].astype(str)+RTO_RVP['Month'].astype(str),
                          RTO_RVP['Year'].astype(str)+"0"+RTO_RVP['Month'].astype(str))
RTO_RVP['Month']=RTO_RVP['Month'].apply(lambda x: int(x))

In [8]:
RTO_RVP_Track=RTO_RVP[(RTO_RVP['Ageing']>3)&(RTO_RVP['Month']>202501)]

In [9]:
UT=UT.drop(['Ageing'],axis=1)
UT['HubName']=UT['HubName'].apply(lambda x:x.upper())
UT=UT.merge(Map,how='left',on=['HubName'])
UT=UT.dropna(subset=['Vendor Name'])
UT['TimeDelta']=dt.now()-pd.to_datetime(UT['CurrentStatusUpdateDateTime'])
UT['Ageing']=UT['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))

In [10]:
UT['Month']=pd.DatetimeIndex(UT['CurrentStatusUpdateDateTime']).month
UT['Year']=pd.DatetimeIndex(UT['CurrentStatusUpdateDateTime']).year
UT=UT.drop(['TimeDelta'],axis=1)
UT['Month']=UT['Month'].astype(str)
UT['Length']=UT['Month'].apply(lambda x:len(x))
UT['Month']=np.where(UT['Length']==2,UT['Year'].astype(str)+UT['Month'].astype(str),
                     UT['Year'].astype(str)+"0"+UT['Month'].astype(str))
UT['Month']=UT['Month'].apply(lambda x: int(x))
UT_Track=UT[(UT['Ageing']>3)&(UT['Month']>202501)]

In [14]:
New3.columns

Index(['Unnamed: 0', 'ShipmentId', ' Status', ' TotalPrice', 'In Cr',
       ' DeliveryHub', ' CurrentHub', 'Mappped hub', 'Asset', 'Owner',
       'Remarks', ' LatestUpdateTime', ' BagId', ' BagStatus', 'ageing',
       'Month', ' NumberOfAttempts', 'RTO DATE', 'Vertical', 'source', 'Zone',
       'GM', 'RM', 'AM / POC', 'seller_id', 'Eagle Eye Remarks Update-25TH',
       'Issue Type', 'Ticket No', 'Amout Recovery Status (Lost)',
       'Recovery Status', 'Hub Reasn Code (FST)'],
      dtype='object')

In [15]:
Map.columns=['cl_current_hub_name','Vendor Name'] #vlookup current hub & vendor name 
New1 = New1.dropna(subset=['cl_current_hub_name']) #current hub NA/Blanks Removed
New1['cl_current_hub_name']=New1['cl_current_hub_name'].apply(lambda x:x.upper()) #Uppercase Currenthub
#New1=New1.merge(Map,how='left',on=['current_hub'])
New1 = New1.merge(Map, how='left', on=['cl_current_hub_name'], suffixes=('', '_map')) #vlookup clcurrenthub & Vendor Name
New1_Track=New1[(New1['Pendency/BRSNR']=='Pendency') & (New1['Vendor Name'].isna()==False)] # Keep Pendeny @C-Pendency/BRSNR , Remove NA
Map.columns=['current_hub','Vendor Name']

New2 = New2.dropna(subset=['current_hub'])
New2['current_hub']=New2['current_hub'].apply(lambda x:x.upper())
New2=New2.merge(Map,how='left',on=['current_hub'], suffixes=('', '_map'))
New2_Temp=New2[(New2['review']=='DH pendency') | (New2['review']=='MP pendency') | (New2['review']=='DH Pendency') | (New2['review']=='MP Pendency')] #Fliter and keep only 
New2_Track=New2_Temp[(New2_Temp['Vendor Name'].isna()==False)]
Map.columns=['Mapped hub','Vendor Name']

# New3['Mappped hub'] = New3['Mappped hub'].str.upper()
# New3 = New3.merge(Map, how='left', on='Mappped hub', suffixes=('', '_map'))
# New3_Track = New3[New3['Vendor Name'].notna()]

In [17]:
RTO_RVP_TIDs = pd.DataFrame(RTO_RVP_Track['ShipmentId'])
UT_TIDs = pd.DataFrame(UT_Track['ShipmentId'])
New1_TIDs = pd.DataFrame(New1_Track['vendor_tracking_id'])
New2_TIDs = pd.DataFrame(New2_Track['vendor_tracking_id'])
# New3_TIDs = pd.DataFrame(New3_Track['ShipmentId'])

In [18]:
New1_TIDs.columns = ['ShipmentId']
New2_TIDs.columns = ['ShipmentId']

In [19]:
Track = pd.concat([RTO_RVP_TIDs, UT_TIDs, New1_TIDs, New2_TIDs]) #Saturday
Track = Track.drop_duplicates()


In [20]:
Track.to_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\All_Multitrack_TIDs.csv")

In [21]:
Automa=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\data_Automa.csv")
Manual=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Multitracking.csv")

In [22]:
Gunti_Multitracked = Manual[Manual['ShipmentId'].isin(New3['ShipmentId'])].drop_duplicates(subset=['ShipmentId']) #saturday
Gunti_Multitracked

,ShipmentId,TierType,Type,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
0,FMPC4721219136,ECONOMY,COD,NaN,NORMAL_FORWARD,783,395010,Expected,Ahmedabad_01,MotherHub_CHG,...,0,Marked_As_RTO,NaN,NaN,NaN,ERAICHG-3534082,IN_TRANSIT,230135098,OD434115722398857100,Samah Embellished Embroidered Bollywood Net S...
1,FMPR0712426802,REGULAR,RVP,NaN,PICKUP_ONLY,233,122503,Undelivered_Not_Attended,sanpka 01,IdentifyPlusMandelaODH_MND,...,1,NaN,RVP,NaN,NaN,Egur_san-2564087,CLOSED,NaN,OD334138260974276100,GALAXY Fruit & Nut Bars.
2,FMPR0711902939,REGULAR,RVP,NaN,PICKUP_ONLY,155,122503,Undelivered_Not_Attended,sanpka 01,Motherhub_CUT,...,1,NaN,NaN,NaN,NaN,EGGNRC-2565443,CLOSED,230238465,OD334119916537835100,Insight Flawless Matte HD Finishing Loose Comp...
3,FMPC4710502861,ECONOMY,COD,NaN,NORMAL_FORWARD,350,741249,Undelivered_Order_Rejected_By_Customer,Kalash_East_Haringhata_FC_1,CABTHaladiapadarODH_BMP,...,2,Marked_As_RTO,NaN,NaN,NaN,EBBIHLD-1640554,TERMINATED,228863449,OD334124086335534100,Renee Midnight Kohl Black Kajal Pencil - Water...
4,FMPR0709494492,REGULAR,RVP,NaN,PICKUP_ONLY,949,741249,Undelivered_Not_Attended,Kalash_East_Haringhata_FC_1,ChhatarpurMDH_DTO,...,6,NaN,Updated by CRON,NaN,NaN,NaN,NaN,NaN,OD334071844744840100,KILLER Slim Men Blue Jeans.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112057,FMPC4705000380,ECONOMY,COD,NaN,NORMAL_FORWARD,7009,302039,Undelivered_Not_Attended,VKIAMPHub_JAI,SutiharaMDH_SMI,...,0,Marked_As_RTO,NaN,NaN,NaN,PH-B4-JAIBTS-15243781,TERMINATED,228064461,OD334106757507956100,ENTWINO Intel Core i5 (8 GB / 500 GB / Windows...
112059,FMPC4705398376,ECONOMY,COD,NaN,NORMAL_FORWARD,252,711322,Undelivered_Order_Rejected_By_Customer,HowrahMPHub_KOL,IdentifyPlusMulMDH_MUL,...,1,Marked_As_RTO,NaN,NaN,NaN,EMDHMUL-1045949,TERMINATED,228941546,OD434111077600829100,BR WORLD COLLECTION Baby Girls Midi/Knee Lengt...
112061,FMPC4705568776,ECONOMY,COD,NaN,NORMAL_FORWARD,23058,302026,Undelivered_Order_Rejected_By_Customer,Jaipur Sanganer NLFC 01,BuhanaMDH_PIL,...,1,Marked_As_RTO,Updated by CRON,NaN,NaN,EBUHPIL-271174,TERMINATED,228258726,OD334111894636269100,Motorola Edge 50 Fusion (Marshmallow Blue 256...
112062,FMPC4705262678,ECONOMY,COD,NaN,NORMAL_FORWARD,10018,124103,Undelivered_Order_Rejected_By_Customer,Farrukhnagar BTS Warehouse,GhalluMDH_GLU,...,2,Marked_As_RTO,NaN,NaN,NaN,EGHAGLU-326229,TERMINATED,228541123,OD434111398699727100,Apple AirPods(2nd gen) with Charging Case Blue...


In [23]:
Gunti_Multitracked.to_excel("Gunti_Multitracked.xlsx", index=False)#saturday


In [24]:
Map.columns=[' CurrentHub','Vendor Name']
Automa[' CurrentHub']=np.where(Automa[' CurrentHub'].isna()==True,"Dummy",Automa[' CurrentHub'])
Automa[' CurrentHub']=Automa[' CurrentHub'].apply(lambda x:x.upper())
#Automa=Automa.merge(Map,how='left',on=[' CurrentHub'])
Automa = Automa.merge(Map, how='left', on=' CurrentHub')
Flag=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Flag.csv")
Automa[' Status']=np.where(Automa[' Status'].isna()==True,"Dummy",Automa[' Status'])
Automa[' Status']=Automa[' Status'].apply(lambda x:x.upper())
Automa=Automa.merge(Flag,how='left',on=[' Status'])
Automa['Remarks']=np.where(Automa['Vendor Name'].isna()==True,'Clear',
                          np.where(Automa['Flag']==1,'Clear',
                                  np.where((Automa[' BagStatus']=='IN_TRANSIT') | (Automa[' BagStatus']=='OPENED') | (Automa[' BagStatus']=='REACHED'),'Clear','To be considered')))
Automa=Automa.drop(['Flag'],axis=1)

In [25]:
Map.columns=[' CurrentHub','Vendor Name']
Manual[' CurrentHub']=np.where(Manual[' CurrentHub'].isna()==True,"Dummy",Manual[' CurrentHub'])
Manual[' CurrentHub']=Manual[' CurrentHub'].apply(lambda x:x.upper())
#Manual=Manual.merge(Map,how='left',on=[' CurrentHub'])
Manual = Manual.merge(Map, how='left', on=' CurrentHub', suffixes=('', '_MAP'))
Flag=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Flag.csv")
Flag.columns=[' Status','Flag']
Manual[' Status']=np.where(Manual[' Status'].isna()==True,"Dummy",Manual[' Status'])
Manual[' Status']=Manual[' Status'].apply(lambda x:x.upper())
Manual=Manual.merge(Flag,how='left',on=[' Status'])
Manual['Remarks']=np.where(Manual['Vendor Name'].isna()==True,'Clear',
                          np.where(Manual['Flag']==1,'Clear',
                                  np.where((Manual[' BagStatus']=='IN_TRANSIT') | (Manual[' BagStatus']=='OPENED') | (Manual[' BagStatus']=='REACHED'),'Clear','To be considered')))
Manual=Manual.drop(['Flag'],axis=1)

In [26]:
UT_TIDs['Flag'] = 1
UT_TIDs = UT_TIDs.drop_duplicates()

UT_TIDs.columns = ['ShipmentId', 'Flag'] # Ensure correct column names in UT_TIDs

Automa = Automa.merge(UT_TIDs, how='left', on='ShipmentId', suffixes=('_automa', '_ut')) # Merge Automa with UT_TIDs on 'ShipmentId'

Manual = Manual.merge(UT_TIDs, how='left', on='ShipmentId') # Merge Manual with UT_TIDs on 'ShipmentId' instead of 'Tracking Id'
#Manual['Remarks'] = np.nan


Automa['Remarks'] = np.where((Automa['Flag'] == 1) & (Automa[' Status'] != 'UNTRACEABLE'), 'Clear', Automa['Remarks'])
Manual['Remarks'] = np.where((Manual['Flag'] == 1) & (Manual[' Status'] != 'UNTRACEABLE'), 'Clear', Manual['Remarks']) # Apply remarks condition


Automa = Automa.drop(['Flag'], axis=1)  # Drop 'Flag' column after merging
Manual = Manual.drop(['Flag'], axis=1)



In [27]:
Automa_Map=Automa[['ShipmentId','Remarks']]
Manual_Map=Manual[['ShipmentId','Remarks']]
Manual_Map.columns=['ShipmentId','Remarks2']
RTO_RVP=RTO_RVP.merge(Automa_Map,how='left',on='ShipmentId')
RTO_RVP=RTO_RVP.merge(Manual_Map,how='left',on='ShipmentId')

In [28]:
UT=UT.merge(Automa_Map,how='left',on='ShipmentId')
UT=UT.merge(Manual_Map,how='left',on='ShipmentId')

In [29]:
RTO_RVP['Final_Remarks']=np.where(RTO_RVP['Remarks'].isna()==True,RTO_RVP['Remarks2'],RTO_RVP['Remarks'])
RTO_RVP=RTO_RVP.drop(['Remarks','Remarks2'],axis=1)
UT['Final_Remarks']=np.where(UT['Remarks'].isna()==True,UT['Remarks2'],UT['Remarks'])
UT=UT.drop(['Remarks','Remarks2'],axis=1)

In [30]:
RTO_RVP_Final=RTO_RVP[RTO_RVP['Final_Remarks']!='Clear']
RTO_RVP_Final=RTO_RVP_Final[(RTO_RVP_Final['Month']>202501)]
UT_Final=UT[UT['Final_Remarks']!='Clear']
UT_Final=UT_Final[UT_Final['CurrentShipmentStatus']=='Untraceable']
UT_Final=UT_Final[(UT_Final['Month']>202501)]
Automa=Automa[Automa['Remarks']!='Clear']
Manual=Manual[Manual['Remarks']!='Clear']

In [31]:
RTO_RVP_Final['LastUpdatDateTime']=np.where(RTO_RVP_Final['TypeOfShipment']=="RTO",RTO_RVP_Final['RTOCreatedDateTime'],RTO_RVP_Final['LastUpdatDateTime'])
RTO_RVP_Final=RTO_RVP_Final[['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']]
RTO_RVP_Final.columns=['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']
UT_Final=UT_Final[['ShipmentId','HubName','CurrentShipmentStatus','CurrentStatusUpdateDateTime','ShipmentAmount','CurrentShipmentStatus','Vendor Name','Month','Ageing']]
UT_Final.columns=['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']

In [32]:
# Attempt to parse with `errors='coerce'`
Manual[' LatestUpdateTime'] = pd.to_datetime(Manual[' LatestUpdateTime'], errors='coerce')

# Check for unparseable entries
invalid_entries = Manual[Manual[' LatestUpdateTime'].isna()]
if not invalid_entries.empty:
    print("Unparseable datetime entries found:", invalid_entries)

# Proceed if column is clean
Manual['TimeDelta'] = dt.now() - Manual[' LatestUpdateTime']
Manual['Ageing'] = Manual['TimeDelta'].apply(lambda x: float(x.days) + float(x.seconds) / (60 * 60 * 24))
Manual['Month'] = Manual[' LatestUpdateTime'].dt.month.astype(str)
Manual['Year'] = Manual[' LatestUpdateTime'].dt.year.astype(str)


In [33]:
Automa['Month']=pd.DatetimeIndex(Automa[' LatestUpdateTime']).month
Automa['Year']=pd.DatetimeIndex(Automa[' LatestUpdateTime']).year
Automa['TimeDelta']=dt.now()-pd.to_datetime(Automa[' LatestUpdateTime'])
Automa['Ageing']=Automa['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))
Automa['Month']=Automa['Month'].astype(str)
Automa['Length']=Automa['Month'].apply(lambda x:len(x))
Automa['Month']=np.where(Automa['Length']==2,Automa['Year'].astype(str)+Automa['Month'].astype(str),
                         Automa['Year'].astype(str)+"0"+Automa['Month'].astype(str))
Manual['Month']=pd.DatetimeIndex(Manual[' LatestUpdateTime']).month
Manual['Year']=pd.DatetimeIndex(Manual[' LatestUpdateTime']).year
Manual['TimeDelta']=dt.now()-pd.to_datetime(Manual[' LatestUpdateTime'])
Manual['Ageing']=Manual['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))
Manual['Month']=Manual['Month'].astype(str)
Manual['Length']=Manual['Month'].apply(lambda x:len(x))
Manual['Month']=np.where(Manual['Length']==2,Manual['Year'].astype(str)+Manual['Month'].astype(str),
                         Manual['Year'].astype(str)+"0"+Manual['Month'].astype(str))

In [34]:
# Remove leading and trailing spaces from column names
Automa.columns = Automa.columns.str.strip()
Manual.columns = Manual.columns.str.strip()

# Select relevant columns
Automa = Automa[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                 'TotalPrice', 'Vendor Name', 'Month', 'Ageing', 'DeliveryHub']]

Manual = Manual[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                 'TotalPrice', 'Vendor Name', 'Month', 'Ageing', 'DeliveryHub']]

# Concatenate both DataFrames
Tracking = pd.concat([Automa, Manual], ignore_index=True)


C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_3204\768821000.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Tracking = pd.concat([Automa, Manual], ignore_index=True)


In [35]:
UT_Map=UT[['ShipmentId']]
UT_Map=UT_Map.drop_duplicates()
UT_Map['Flag']=1
Tracking=Tracking.merge(UT_Map,how='left',on='ShipmentId')
Tracking1=sqldf("select * from Tracking where DeliveryHub like '%Grocery%'")
Tracking2=sqldf("select * from Tracking where DeliveryHub not like '%Grocery%'")
Tracking1['Type']='GTNL'
Tracking2['Type']=np.where(Tracking2['Flag']==1,'Untraceable',
                         np.where(Tracking2['ShipmentId'].str.slice(3,4)=="R","RVP","RTO"))
Tracking=pd.concat([Tracking1,Tracking2])

In [36]:


# Now select the required columns
Tracking = Tracking[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                     'TotalPrice', 'Type', 'Vendor Name', 'Month', 'Ageing']]

# Rename the columns
Tracking.columns = ['ShipmentId', 'HubName', 'CurrentStatus', 'LastUpdatDateTime', 
                    'ShipmentValue', 'TypeOfShipment', 'Vendor Name', 'Month', 'Ageing']

# Convert 'Month' column to numeric, handling errors
Tracking['Month'] = pd.to_numeric(Tracking['Month'], errors='coerce')

# Drop rows with invalid or missing 'Month' values
Tracking = Tracking.dropna(subset=['Month'])

# Convert 'Month' to integers
Tracking['Month'] = Tracking['Month'].astype(int)

# Filter rows where 'Month' > 202411
Tracking = Tracking[Tracking['Month'] > 202501]


In [37]:
Final=pd.concat([RTO_RVP_Final,UT_Final])
Final=pd.concat([Final,Tracking])
Final=Final.drop_duplicates(subset=['ShipmentId'])
Final['Slab']=np.where(Final['Ageing']<=2,"0 to 2",
                      np.where(Final['Ageing']<=5,"2 to 5",">5"))
#New1_TIDs=pd.concat([New1_TIDs,Prexo_TIDs])
New1_TIDs['Model']='FA'
Final=Final.merge(New1_TIDs,how='left',on='ShipmentId')
Final['Model']=np.where(Final['Model'].isna()==True,"Others",Final['Model'])

In [38]:
Debit = pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Debit_data.csv", encoding="ISO-8859-1")
Debit=Debit.drop_duplicates()
Debit['ShipmentId']=Debit['ShipmentId'].apply(lambda x:x.upper())
Final=Final.merge(Debit,how='left',on='ShipmentId')
Final=Final[Final['Flag']!=1]
Final=Final.drop(['Flag'],axis=1)
Final.to_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Cron_Data.csv") ##end